시군구 파일에서 지역 통합시키는 전처리

In [41]:
import pandas as pd

# 1. 시군구코드 기준 원본 파일 불러오기
df = pd.read_csv("city_county_code.csv", dtype={"주소(시군구)": int})

# 2. 컬럼명 공백 제거
df.columns = df.columns.str.strip()

# 3. 통합 범위 정의 (start ~ end, 통합이름)
merge_rules = [
    (11110, 11740, "서울특별시"),
    (26110, 26710, "부산광역시"),
    (27110, 27720, "대구광역시"),
    (28110, 28720, "인천광역시"),
    (29110, 29200, "광주광역시"),
    (30110, 30230, "대전광역시"),
    (31110, 31710, "울산광역시"),
    (41110, 41117, "경기도 수원시"),
    (41130, 41135, "경기도 성남시"),
    (41170, 41173, "경기도 안양시"),
    (41190, 41196, "경기도 부천시"),
    (41270, 41273, "경기도 안산시"),
    (41280, 41287, "경기도 고양시"),
    (41460, 41465, "경기도 용인시"),
    (43110, 43114, "충청북도 청주시"),
    (44130, 44133, "충청남도 천안시"),
    (45110, 45113, "전라북도 전주시"),
    (47110, 47113, "경상북도 포항시"),
    (48120, 48129, "경상남도 창원시"),
]

# 4. 통합 시도명 적용 함수
def apply_merge_name(code):
    for start, end, name in merge_rules:
        if start <= code <= end:
            return name
    return None

# 5. 통합된 이름 칼럼 추가
df["통합시도명"] = df["주소(시군구)"].apply(apply_merge_name)

# 6. 최종 시도명 결정
df["최종시도명"] = df.apply(
    lambda row: row["통합시도명"] if pd.notna(row["통합시도명"])
    else f"{row['시도명']} {row['시군구명']}".strip(),
    axis=1
)

# 7. 중복 제거 및 정렬
df_result = df[["최종시도명"]].drop_duplicates().sort_values(by="최종시도명").reset_index(drop=True)

# 8. 저장
df_result.to_csv("integrated_region_list.csv", index=False, encoding="utf-8-sig")

천식 환자 & (외래)천식 환자 그룹화

In [42]:
import pandas as pd

# CSV 파일 읽기 (일반 천식 환자)
df = pd.read_csv("patient(asthma).csv", dtype={"요양개시연월": str, "주소(시군구)": str})

# 필요한 컬럼만 선택
df = df[["요양개시연월", "주소(시군구)", "진료에피소드 건수"]]

# 숫자형으로 변환
df["진료에피소드 건수"] = pd.to_numeric(df["진료에피소드 건수"], errors="coerce")

# 그룹화 및 합산
grouped = df.groupby(["요양개시연월", "주소(시군구)"], as_index=False)["진료에피소드 건수"].sum()

# 저장
grouped.to_csv("total_treatments(asthma patients).csv", index=False, encoding="utf-8-sig")

In [43]:
import pandas as pd

# CSV 파일 읽기 (외래 천식 환자)
df = pd.read_csv("outpatient(asthma).csv", dtype={"요양개시연월": str, "주소(시군구)": str})

# 필요한 컬럼만 선택
df = df[["요양개시연월", "주소(시군구)", "진료에피소드 건수"]]

# 숫자형으로 변환
df["진료에피소드 건수"] = pd.to_numeric(df["진료에피소드 건수"], errors="coerce")

# 그룹화 및 합산
grouped = df.groupby(["요양개시연월", "주소(시군구)"], as_index=False)["진료에피소드 건수"].sum()

# 저장
grouped.to_csv("total_treatments(asthma outpatients).csv", index=False, encoding="utf-8-sig")

합계와 시군구 코드 병합

In [44]:
import pandas as pd

# 파일 불러오기
df_main = pd.read_csv("total_treatments(asthma patients).csv")
df_code = pd.read_csv("city_county_code.csv")

# 주소(시군구) 컬럼명을 맞추고 둘 다 문자열로 변환
df_main["주소(시군구)"] = df_main["주소(시군구)"].astype(str)
df_code["주소(시군구)"] = df_code["주소(시군구)"].astype(str)

# 열 이름 통일
df_code = df_code.rename(columns={"시군구코드": "주소(시군구)"})

# 병합
df_merged = pd.merge(df_main, df_code, on="주소(시군구)", how="left")

# CSV로 저장
df_merged.to_csv("merged_asthma_patients.csv", index=False, encoding="utf-8-sig")

In [45]:
import pandas as pd

# 파일 불러오기 (먼저 로딩)
df_main = pd.read_csv("total_treatments(asthma outpatients).csv")
df_code = pd.read_csv("city_county_code.csv")

# 주소(시군구) 컬럼명을 맞추고 둘 다 문자열로 변환
df_main["주소(시군구)"] = df_main["주소(시군구)"].astype(str)
df_code["주소(시군구)"] = df_code["주소(시군구)"].astype(str)

# 열 이름 통일
df_code = df_code.rename(columns={"시군구코드": "주소(시군구)"})

# 병합
df_merged = pd.merge(df_main, df_code, on="주소(시군구)", how="left")

# CSV로 저장
df_merged.to_csv("merged_asthma_outpatients.csv", index=False, encoding="utf-8-sig")

최종 천식 & (외래)천식 환자 종합

In [46]:
import pandas as pd

# 데이터 불러오기
df = pd.read_csv("merged_asthma_patients.csv", dtype={"주소(시군구)": str})

# 통합 매핑 정의
merge_rules = [
    (11110, 11740, "서울특별시"),
    (26110, 26710, "부산광역시"),
    (27110, 27720, "대구광역시"),
    (28110, 28720, "인천광역시"),
    (29110, 29200, "광주광역시"),
    (30110, 30230, "대전광역시"),
    (31110, 31710, "울산광역시"),
    (41110, 41117, "경기도 수원시"),
    (41130, 41135, "경기도 성남시"),
    (41170, 41173, "경기도 안양시"),
    (41190, 41196, "경기도 부천시"),
    (41270, 41273, "경기도 안산시"),
    (41280, 41287, "경기도 고양시"),
    (41460, 41465, "경기도 용인시"),
    (43110, 43114, "충청북도 청주시"),
    (44130, 44133, "충청남도 천안시"),
    (45110, 45113, "전라북도 전주시"),
    (47110, 47113, "경상북도 포항시"),
    (48120, 48129, "경상남도 창원시"),
]

# 주소(시군구) 컬럼을 정수로 변환
df["주소(시군구)"] = df["주소(시군구)"].astype(int)

# 새로운 '시도명_통합' 컬럼 만들기
def map_region(code, orig_sido, orig_sigungu):
    for start, end, name in merge_rules:
        if start <= code <= end:
            return name
    return f"{orig_sido} {orig_sigungu}".strip()

df["시도명_통합"] = df.apply(
    lambda row: map_region(row["주소(시군구)"], row["시도명"], row["시군구명"]),
    axis=1
)

# 요양개시연월 + 시도명_통합 기준으로 집계
df_grouped = df.groupby(["요양개시연월", "시도명_통합"])["진료에피소드 건수"].sum().reset_index()

# 컬럼명 정리
df_grouped.rename(columns={"시도명_통합": "지역"}, inplace=True)

# 결과 저장
df_grouped.to_csv("final_asthma_patient.csv", index=False, encoding="utf-8-sig")

In [47]:
import pandas as pd

# 데이터 불러오기
df = pd.read_csv("merged_asthma_outpatients.csv", dtype={"주소(시군구)": str})

# 통합 매핑 정의
merge_rules = [
    (11110, 11740, "서울특별시"),
    (26110, 26710, "부산광역시"),
    (27110, 27720, "대구광역시"),
    (28110, 28720, "인천광역시"),
    (29110, 29200, "광주광역시"),
    (30110, 30230, "대전광역시"),
    (31110, 31710, "울산광역시"),
    (41110, 41117, "경기도 수원시"),
    (41130, 41135, "경기도 성남시"),
    (41170, 41173, "경기도 안양시"),
    (41190, 41196, "경기도 부천시"),
    (41270, 41273, "경기도 안산시"),
    (41280, 41287, "경기도 고양시"),
    (41460, 41465, "경기도 용인시"),
    (43110, 43114, "충청북도 청주시"),
    (44130, 44133, "충청남도 천안시"),
    (45110, 45113, "전라북도 전주시"),
    (47110, 47113, "경상북도 포항시"),
    (48120, 48129, "경상남도 창원시"),
]

# 주소(시군구) 컬럼을 정수로 변환
df["주소(시군구)"] = df["주소(시군구)"].astype(int)

# 새로운 '시도명_통합' 컬럼 만들기
def map_region(code, orig_sido, orig_sigungu):
    for start, end, name in merge_rules:
        if start <= code <= end:
            return name
    return f"{orig_sido} {orig_sigungu}".strip()

df["시도명_통합"] = df.apply(
    lambda row: map_region(row["주소(시군구)"], row["시도명"], row["시군구명"]),
    axis=1
)

# 요양개시연월 + 시도명_통합 기준으로 집계
df_grouped = df.groupby(["요양개시연월", "시도명_통합"])["진료에피소드 건수"].sum().reset_index()

# 컬럼명 정리
df_grouped.rename(columns={"시도명_통합": "지역"}, inplace=True)

# 결과 저장
df_grouped.to_csv("final_asthma_outpatient.csv", index=False, encoding="utf-8-sig")

비염 환자 & (외래)비염 환자 그룹화

In [48]:
import pandas as pd

# CSV 파일 읽기 (일반 천식 환자)
df = pd.read_csv("patient(rhinitis).csv", dtype={"요양개시연월": str, "주소(시군구)": str})

# 필요한 컬럼만 선택
df = df[["요양개시연월", "주소(시군구)", "진료에피소드 건수"]]

# 숫자형으로 변환
df["진료에피소드 건수"] = pd.to_numeric(df["진료에피소드 건수"], errors="coerce")

# 그룹화 및 합산
grouped = df.groupby(["요양개시연월", "주소(시군구)"], as_index=False)["진료에피소드 건수"].sum()

# 저장
grouped.to_csv("total_treatments(rhinitis patients).csv", index=False, encoding="utf-8-sig")

In [49]:
import pandas as pd

# CSV 파일 읽기 (외래 천식 환자)
df = pd.read_csv("outpatient(rhinitis).csv", dtype={"요양개시연월": str, "주소(시군구)": str})

# 필요한 컬럼만 선택
df = df[["요양개시연월", "주소(시군구)", "진료에피소드 건수"]]

# 숫자형으로 변환
df["진료에피소드 건수"] = pd.to_numeric(df["진료에피소드 건수"], errors="coerce")

# 그룹화 및 합산
grouped = df.groupby(["요양개시연월", "주소(시군구)"], as_index=False)["진료에피소드 건수"].sum()

# 저장
grouped.to_csv("total_treatments(rhinitis outpatients).csv", index=False, encoding="utf-8-sig")

합계와 시군구 코드 병합

In [50]:
import pandas as pd

# 파일 불러오기
df_main = pd.read_csv("total_treatments(rhinitis patients).csv")
df_code = pd.read_csv("city_county_code.csv")

# 주소(시군구) 컬럼명을 맞추고 둘 다 문자열로 변환
df_main["주소(시군구)"] = df_main["주소(시군구)"].astype(str)
df_code["주소(시군구)"] = df_code["주소(시군구)"].astype(str)

# 열 이름 통일
df_code = df_code.rename(columns={"시군구코드": "주소(시군구)"})

# 병합
df_merged = pd.merge(df_main, df_code, on="주소(시군구)", how="left")

# CSV로 저장
df_merged.to_csv("merged_rhinitis_patients.csv", index=False, encoding="utf-8-sig")

In [51]:
import pandas as pd

# 파일 불러오기
df_main = pd.read_csv("total_treatments(rhinitis outpatients).csv")
df_code = pd.read_csv("city_county_code.csv")

# 주소(시군구) 컬럼명을 맞추고 둘 다 문자열로 변환
df_main["주소(시군구)"] = df_main["주소(시군구)"].astype(str)
df_code["주소(시군구)"] = df_code["주소(시군구)"].astype(str)

# 열 이름 통일
df_code = df_code.rename(columns={"시군구코드": "주소(시군구)"})

# 병합
df_merged = pd.merge(df_main, df_code, on="주소(시군구)", how="left")

# CSV로 저장
df_merged.to_csv("merged_rhinitis_outpatients.csv", index=False, encoding="utf-8-sig")

최종 비염 & (외래)비염 환자 종합

In [52]:
import pandas as pd

# 데이터 불러오기
df = pd.read_csv("merged_rhinitis_patients.csv", dtype={"주소(시군구)": str})

# 통합 매핑 정의
merge_rules = [
    (11110, 11740, "서울특별시"),
    (26110, 26710, "부산광역시"),
    (27110, 27720, "대구광역시"),
    (28110, 28720, "인천광역시"),
    (29110, 29200, "광주광역시"),
    (30110, 30230, "대전광역시"),
    (31110, 31710, "울산광역시"),
    (41110, 41117, "경기도 수원시"),
    (41130, 41135, "경기도 성남시"),
    (41170, 41173, "경기도 안양시"),
    (41190, 41196, "경기도 부천시"),
    (41270, 41273, "경기도 안산시"),
    (41280, 41287, "경기도 고양시"),
    (41460, 41465, "경기도 용인시"),
    (43110, 43114, "충청북도 청주시"),
    (44130, 44133, "충청남도 천안시"),
    (45110, 45113, "전라북도 전주시"),
    (47110, 47113, "경상북도 포항시"),
    (48120, 48129, "경상남도 창원시"),
]

# 주소(시군구) 컬럼을 정수로 변환
df["주소(시군구)"] = df["주소(시군구)"].astype(int)

# 새로운 '시도명_통합' 컬럼 만들기
def map_region(code, orig_sido, orig_sigungu):
    for start, end, name in merge_rules:
        if start <= code <= end:
            return name
    return f"{orig_sido} {orig_sigungu}".strip()

df["시도명_통합"] = df.apply(
    lambda row: map_region(row["주소(시군구)"], row["시도명"], row["시군구명"]),
    axis=1
)

# 요양개시연월 + 시도명_통합 기준으로 집계
df_grouped = df.groupby(["요양개시연월", "시도명_통합"])["진료에피소드 건수"].sum().reset_index()

# 컬럼명 정리
df_grouped.rename(columns={"시도명_통합": "지역"}, inplace=True)

# 결과 저장
df_grouped.to_csv("final_rhinitis_patient.csv", index=False, encoding="utf-8-sig")

In [53]:
import pandas as pd

# 데이터 불러오기
df = pd.read_csv("merged_rhinitis_outpatients.csv", dtype={"주소(시군구)": str})

# 통합 매핑 정의
merge_rules = [
    (11110, 11740, "서울특별시"),
    (26110, 26710, "부산광역시"),
    (27110, 27720, "대구광역시"),
    (28110, 28720, "인천광역시"),
    (29110, 29200, "광주광역시"),
    (30110, 30230, "대전광역시"),
    (31110, 31710, "울산광역시"),
    (41110, 41117, "경기도 수원시"),
    (41130, 41135, "경기도 성남시"),
    (41170, 41173, "경기도 안양시"),
    (41190, 41196, "경기도 부천시"),
    (41270, 41273, "경기도 안산시"),
    (41280, 41287, "경기도 고양시"),
    (41460, 41465, "경기도 용인시"),
    (43110, 43114, "충청북도 청주시"),
    (44130, 44133, "충청남도 천안시"),
    (45110, 45113, "전라북도 전주시"),
    (47110, 47113, "경상북도 포항시"),
    (48120, 48129, "경상남도 창원시"),
]

# 주소(시군구) 컬럼을 정수로 변환
df["주소(시군구)"] = df["주소(시군구)"].astype(int)

# 새로운 '시도명_통합' 컬럼 만들기
def map_region(code, orig_sido, orig_sigungu):
    for start, end, name in merge_rules:
        if start <= code <= end:
            return name
    return f"{orig_sido} {orig_sigungu}".strip()

df["시도명_통합"] = df.apply(
    lambda row: map_region(row["주소(시군구)"], row["시도명"], row["시군구명"]),
    axis=1
)

# 요양개시연월 + 시도명_통합 기준으로 집계
df_grouped = df.groupby(["요양개시연월", "시도명_통합"])["진료에피소드 건수"].sum().reset_index()

# 컬럼명 정리
df_grouped.rename(columns={"시도명_통합": "지역"}, inplace=True)

# 결과 저장
df_grouped.to_csv("final_rhinitis_outpatient.csv", index=False, encoding="utf-8-sig")


아토피 환자 & (외래)아토피 환자 그룹화

In [54]:
import pandas as pd

# CSV 파일 읽기 (일반 아토피 환자)
df = pd.read_csv("patient(atopic).csv", dtype={"요양개시연월": str, "주소(시군구)": str})

# 필요한 컬럼만 선택
df = df[["요양개시연월", "주소(시군구)", "진료에피소드 건수"]]

# 숫자형으로 변환
df["진료에피소드 건수"] = pd.to_numeric(df["진료에피소드 건수"], errors="coerce")

# 그룹화 및 합산
grouped = df.groupby(["요양개시연월", "주소(시군구)"], as_index=False)["진료에피소드 건수"].sum()

# 저장
grouped.to_csv("total_treatments(atopic patients).csv", index=False, encoding="utf-8-sig")

In [55]:
import pandas as pd

# ✅ CSV 파일 읽기 (외래 아토피 환자)
df = pd.read_csv("outpatient(atopic).csv", dtype={"요양개시연월": str, "주소(시군구)": str})

# 필요한 컬럼만 선택
df = df[["요양개시연월", "주소(시군구)", "진료에피소드 건수"]]

# 숫자형으로 변환
df["진료에피소드 건수"] = pd.to_numeric(df["진료에피소드 건수"], errors="coerce")

# 그룹화 및 합산
grouped = df.groupby(["요양개시연월", "주소(시군구)"], as_index=False)["진료에피소드 건수"].sum()

# 저장
grouped.to_csv("total_treatments(atopic outpatients).csv", index=False, encoding="utf-8-sig")

합계와 시군구 코드 병합

In [56]:
import pandas as pd

# 파일 불러오기
df_main = pd.read_csv("total_treatments(atopic patients).csv")
df_code = pd.read_csv("city_county_code.csv")

# 주소(시군구) 컬럼명을 맞추고 둘 다 문자열로 변환
df_main["주소(시군구)"] = df_main["주소(시군구)"].astype(str)
df_code["주소(시군구)"] = df_code["주소(시군구)"].astype(str)

# 열 이름 통일
df_code = df_code.rename(columns={"시군구코드": "주소(시군구)"})

# 병합
df_merged = pd.merge(df_main, df_code, on="주소(시군구)", how="left")

# CSV로 저장
df_merged.to_csv("merged_atopic_patients.csv", index=False, encoding="utf-8-sig")

In [57]:
import pandas as pd

# 파일 불러오기
df_main = pd.read_csv("total_treatments(atopic outpatients).csv")
df_code = pd.read_csv("city_county_code.csv")

# 주소(시군구) 컬럼명을 맞추고 둘 다 문자열로 변환
df_main["주소(시군구)"] = df_main["주소(시군구)"].astype(str)
df_code["주소(시군구)"] = df_code["주소(시군구)"].astype(str)

# 열 이름 통일
df_code = df_code.rename(columns={"시군구코드": "주소(시군구)"})

# 병합
df_merged = pd.merge(df_main, df_code, on="주소(시군구)", how="left")

# CSV로 저장
df_merged.to_csv("merged_atopic_outpatients.csv", index=False, encoding="utf-8-sig")

최종 아토피 & (외래)아토피 환자 종합

In [58]:
import pandas as pd

# 데이터 불러오기
df = pd.read_csv("merged_atopic_patients.csv", dtype={"주소(시군구)": str})

# 통합 매핑 정의
merge_rules = [
    (11110, 11740, "서울특별시"),
    (26110, 26710, "부산광역시"),
    (27110, 27720, "대구광역시"),
    (28110, 28720, "인천광역시"),
    (29110, 29200, "광주광역시"),
    (30110, 30230, "대전광역시"),
    (31110, 31710, "울산광역시"),
    (41110, 41117, "경기도 수원시"),
    (41130, 41135, "경기도 성남시"),
    (41170, 41173, "경기도 안양시"),
    (41190, 41196, "경기도 부천시"),
    (41270, 41273, "경기도 안산시"),
    (41280, 41287, "경기도 고양시"),
    (41460, 41465, "경기도 용인시"),
    (43110, 43114, "충청북도 청주시"),
    (44130, 44133, "충청남도 천안시"),
    (45110, 45113, "전라북도 전주시"),
    (47110, 47113, "경상북도 포항시"),
    (48120, 48129, "경상남도 창원시"),
]

# 주소(시군구) 컬럼을 정수로 변환
df["주소(시군구)"] = df["주소(시군구)"].astype(int)

# 새로운 '시도명_통합' 컬럼 만들기
def map_region(code, orig_sido, orig_sigungu):
    for start, end, name in merge_rules:
        if start <= code <= end:
            return name
    return f"{orig_sido} {orig_sigungu}".strip()

df["시도명_통합"] = df.apply(
    lambda row: map_region(row["주소(시군구)"], row["시도명"], row["시군구명"]),
    axis=1
)

# 요양개시연월 + 시도명_통합 기준으로 집계
df_grouped = df.groupby(["요양개시연월", "시도명_통합"])["진료에피소드 건수"].sum().reset_index()

# 컬럼명 정리
df_grouped.rename(columns={"시도명_통합": "지역"}, inplace=True)

# 결과 저장
df_grouped.to_csv("final_atopic_patient.csv", index=False, encoding="utf-8-sig")

In [59]:
import pandas as pd

# 데이터 불러오기
df = pd.read_csv("merged_atopic_outpatients.csv", dtype={"주소(시군구)": str})

# 통합 매핑 정의
merge_rules = [
    (11110, 11740, "서울특별시"),
    (26110, 26710, "부산광역시"),
    (27110, 27720, "대구광역시"),
    (28110, 28720, "인천광역시"),
    (29110, 29200, "광주광역시"),
    (30110, 30230, "대전광역시"),
    (31110, 31710, "울산광역시"),
    (41110, 41117, "경기도 수원시"),
    (41130, 41135, "경기도 성남시"),
    (41170, 41173, "경기도 안양시"),
    (41190, 41196, "경기도 부천시"),
    (41270, 41273, "경기도 안산시"),
    (41280, 41287, "경기도 고양시"),
    (41460, 41465, "경기도 용인시"),
    (43110, 43114, "충청북도 청주시"),
    (44130, 44133, "충청남도 천안시"),
    (45110, 45113, "전라북도 전주시"),
    (47110, 47113, "경상북도 포항시"),
    (48120, 48129, "경상남도 창원시"),
]

# 주소(시군구) 컬럼을 정수로 변환
df["주소(시군구)"] = df["주소(시군구)"].astype(int)

# 새로운 '시도명_통합' 컬럼 만들기
def map_region(code, orig_sido, orig_sigungu):
    for start, end, name in merge_rules:
        if start <= code <= end:
            return name
    return f"{orig_sido} {orig_sigungu}".strip()

df["시도명_통합"] = df.apply(
    lambda row: map_region(row["주소(시군구)"], row["시도명"], row["시군구명"]),
    axis=1
)

# 요양개시연월 + 시도명_통합 기준으로 집계
df_grouped = df.groupby(["요양개시연월", "시도명_통합"])["진료에피소드 건수"].sum().reset_index()

# 컬럼명 정리
df_grouped.rename(columns={"시도명_통합": "지역"}, inplace=True)

# 결과 저장
df_grouped.to_csv("final_atopic_outpatient.csv", index=False, encoding="utf-8-sig")

각 질환자 병합

In [60]:
import pandas as pd

def merge_patient_data(file_outpatient, file_total, output_name):
    # CSV 불러오기
    df_out = pd.read_csv(file_outpatient)
    df_tot = pd.read_csv(file_total)
    
    # 컬럼이름이 정확히 같은지 확인 후 일치시킴
    common_cols = ["요양개시연월", "지역", "진료에피소드 건수"]
    df_out = df_out[common_cols]
    df_tot = df_tot[common_cols]

    # 두 파일 병합 및 건수 합산
    df_combined = pd.concat([df_out, df_tot])
    df_grouped = df_combined.groupby(["요양개시연월", "지역"], as_index=False)["진료에피소드 건수"].sum()

    # 결과 저장
    df_grouped.to_csv(output_name, index=False, encoding="utf-8-sig")

# 5. 각 질병별로 병합 실행
merge_patient_data("final_asthma_outpatient.csv", "final_asthma_patient.csv", "final_asthma_total.csv")
merge_patient_data("final_atopic_outpatient.csv", "final_atopic_patient.csv", "final_atopic_total.csv")
merge_patient_data("final_rhinitis_outpatient.csv", "final_rhinitis_patient.csv", "final_rhinitis_total.csv")

모든 질환자 수 병합

In [61]:
import pandas as pd

# 각 질환 합산 파일 불러오기
df_asthma = pd.read_csv("final_asthma_total.csv")
df_atopy = pd.read_csv("final_atopic_total.csv")
df_rhinitis = pd.read_csv("final_rhinitis_total.csv")

# 모두 하나로 concat
df_all = pd.concat([df_asthma, df_atopy, df_rhinitis])

# '요양개시연월' + '시도명' 기준으로 진료에피소드 건수 합산
df_grouped = df_all.groupby(["요양개시연월", "지역"], as_index=False)["진료에피소드 건수"].sum()

# 최종 저장
df_grouped.to_csv("total_respiratory_disease.csv", index=False, encoding="utf-8-sig")

기간 설정 2015.01 ~ 2020.12

In [62]:
import pandas as pd

# 파일 불러오기
df = pd.read_csv("total_respiratory_disease.csv")

# '요양개시연월'을 datetime으로 변환
df["요양개시연월"] = pd.to_datetime(df["요양개시연월"], format="%Y-%m")

# 기간 필터링 (2015-01 ~ 2020-12)
start = "2015-01"
end = "2020-12"
df_filtered = df[(df["요양개시연월"] >= start) & (df["요양개시연월"] <= end)]

# 다시 문자열로 변환 (연월 형식 유지)
df_filtered["요양개시연월"] = df_filtered["요양개시연월"].dt.strftime("%Y-%m")

# 저장
df_filtered.to_csv("respiratory_case_2015_2020.csv", index=False, encoding="utf-8-sig")

C:\Users\080a3\AppData\Local\Temp\ipykernel_33228\3260248886.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["요양개시연월"] = df_filtered["요양개시연월"].dt.strftime("%Y-%m")


열과 행 변환

In [63]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv("respiratory_case_2015_2020.csv", encoding="utf-8-sig")

# 날짜 처리: 자동 형식 추론
df['요양개시연월'] = pd.to_datetime(df['요양개시연월'], errors='coerce')
df['요양개시연월'] = df['요양개시연월'].dt.strftime('%Y.%m')

# 진료 건수 숫자형 변환
df['진료에피소드 건수'] = pd.to_numeric(df['진료에피소드 건수'], errors='coerce')

# 피벗 테이블: 지역별 월간 총합
df_pivot = df.pivot_table(
    index='지역',
    columns='요양개시연월',
    values='진료에피소드 건수',
    aggfunc='sum'
)

# 월 순서대로 정렬
df_pivot = df_pivot[sorted(df_pivot.columns)]

# 인덱스를 컬럼으로 변환
df_pivot = df_pivot.reset_index()
df_pivot.columns.name = None

# 저장
df_pivot.to_csv("respiratory_case.csv", index=False, encoding="utf-8-sig")

호흡기 질환자수와 미세먼지 시각화 그래프

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지

# 1. 미세먼지 데이터 불러오기
pm_df = pd.read_csv('WhoMergedPm.csv', encoding='utf-8')  # 인코딩 확인 필요

# 2. 호흡기질환자수 데이터 불러오기
resp_df = pd.read_csv('respiratory_case.csv', encoding='utf-8')  # 인코딩 확인 필요

# 3. 미세먼지 월별 컬럼 (지역 제외)
pm_month_cols = [col for col in pm_df.columns if col != '지역']

# 4. 미세먼지 월별 전체 지역 평균 계산
pm_monthly_avg = pm_df[pm_month_cols].mean()

# 5. 호흡기질환자수 월별 컬럼 (지역 제외)
resp_month_cols = [col for col in resp_df.columns if col != '지역']

# 6. 호흡기질환자수 월별 전체 지역 합계 또는 평균 계산
# (합계가 적절하면 sum, 평균이 적절하면 mean 사용)
resp_monthly_sum = resp_df[resp_month_cols].sum()

# 7. 그래프 그리기 (2개를 y축 공유하지 않고 나란히 그리기)

fig, axes = plt.subplots(2, 1, figsize=(20, 10), sharex=True)

# 8. 미세먼지 그래프
axes[0].plot(pm_month_cols, pm_monthly_avg, marker='o', linestyle='-')
axes[0].set_title('2015.03~2020.12 월별 미세먼지 평균 (전체 지역)')
axes[0].set_ylabel('미세먼지 농도 (㎍/㎥)')
axes[0].grid(True)

# 9. 호흡기질환자수 그래프
axes[1].plot(resp_month_cols, resp_monthly_sum, marker='s', color='r', linestyle='-')
axes[1].set_title('2015.03~2020.12 월별 호흡기 질환자 수 합계 (전체 지역)')
axes[1].set_xlabel('년월')
axes[1].set_ylabel('호흡기 질환자 수')
axes[1].grid(True)

# x축 라벨 회전 및 레이아웃 조정
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()
